In [2]:
import pandas as pd
pd.set_option('mode.copy_on_write', True)
import numpy as np

Create simulated patient data

In [3]:
# Set random seed for reproducibility
np.random.seed(42)

# Initial number of patients
n_patients = 1000

# Creating initial pain scores from ethnographic data
real_pain_scores = np.array([8,8,5,2])
mean_pain_score = np.mean(real_pain_scores) 
std_dev_pain_score = np.std(real_pain_scores)
initial_pain_scores = np.random.normal(std_dev_pain_score, mean_pain_score, n_patients).round().astype(int)
initial_pain_scores = np.clip(initial_pain_scores, 1, 10)

In [4]:
# Generate random ages for patients between 30 and 70
ages = np.random.randint(30,70, size=n_patients) 

# Assign patients randomly to vr therapy or control group
vr_therapy = np.random.choice([0,1], size=n_patients)

In [5]:
# simulate standard of care variables 
prescription_pain_relief = np.random.choice(['yes', 'no'], size=n_patients)
cbt = np.random.choice(['yes', 'no'], size=n_patients)
support_group = np.random.choice(['yes', 'no'], size=n_patients)
physiotherapy = np.random.choice(['yes', 'no'], size=n_patients)
                                        
                                             

In [10]:
# create a dataframe for our simulated patient data
data = {
    'Age': ages,
    'Initial Pain Score': initial_pain_scores,
    'VR Therapy': vr_therapy,
    'Prescription Pain Relief': prescription_pain_relief,
    'CBT': cbt,
    'Support Group': support_group,
    'Physiotherapy': physiotherapy,
    'Group': ['VR Therapy' if i == 1 else 'Control' for i in vr_therapy]
}

patient_df = pd.DataFrame(data)

# df.to_pickle('simulated_clinical_trial_data.pkl')

# Check the first few rowss
patient_df.head()

,Age,Initial Pain Score,VR Therapy,Prescription Pain Relief,CBT,Support Group,Physiotherapy,Group
0,69,5,0,yes,yes,yes,no,Control
1,40,2,0,no,yes,yes,no,Control
2,32,6,0,yes,no,no,no,Control
3,35,10,1,no,no,yes,no,VR Therapy
4,38,1,1,no,yes,yes,yes,VR Therapy


In [11]:
vr_therapy_counts = patient_df['VR Therapy'].value_counts()
vr_therapy_counts

VR Therapy
0    518
1    482
Name: count, dtype: int64

### Estimating VR Effect Size ###
The ethnographic anonymous data provides four patients pain scores before and after trialing a 7 minute 'self care' immersive VR therapy. I'll compute the differences between the pain scores before and after the VR session for each patient, calculate the mean and standard deviation of these differences and use this to calculate Cohen's d (the mean difference divided by the standard deviation of the differences) to determine a reasonable estimate for effect size. 

In [12]:
from statsmodels.stats.power import TTestIndPower

In [14]:
# Determining effect size from ethnographic data
pain_before = np.array([8,8,5,2])
pain_after = np.array([4,1,5,1]) 

# Calculate the differences in scores
pain_differences = pain_before - pain_after

# Calculate the mean and standard deviation of the differences
mean_difference = np.mean(pain_differences)
std_dev_difference = np.std(pain_differences, ddof=1) # numpy advice to provide an unbiased estimator of the variance of the infinite population

# Calculate the effect size
effect_size = mean_difference / std_dev_difference

# Print results
print(f"Mean Difference: {mean_difference}")
print(f"Standard Deviation of Differences: {std_dev_difference}")
print(f"Effect Size: {effect_size}")

Mean Difference: 3.0
Standard Deviation of Differences: 3.1622776601683795
Effect Size: 0.9486832980505138


In [62]:
print(pain_differences)

[4 7 0 1]


In [ ]:
print()

So this is a pretty large effect and based on a TINY sample (95% difference after VR therapy)... not sure this is realistic? Speak to Matthew

In [15]:
# Investigating sample size with an effect of 95% and a power of 80%

# Parameters for power analysis
effect_size = 0.95  # effect size based on ethnographic data
alpha = 0.05  # The common threshold for significance level
power = 0.80  # Desired power aka probability of correctly identifying an effect when there is one 80% of the time - also a medical standard to miss an effect 20% is OK

# Perform power analysis
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=1.0)

# Print the required sample size
print(f"Required sample size per group: {int(np.ceil(sample_size))}")


Required sample size per group: 19


# Let's simulate a trial #

### First we need to set some parameters for our trial simulation, including, defining the following variables: ###
- Number of patients per group: this is pulled from our earlier power calculation 
- Initial pain scores (mean): derived from the mean pain score from ethnographic data
- Initial pain scores (std.dev): derived from the std_dev of pain scores from ethnographic data
- Number of simulations: how many times we want to run the trial 

### We also need to define and take account of (and can adjust) the expected pain reduction from each SOC care treatment ###
- CBT 
- Physio 
- Pain relief 
- Support group

In [30]:

# Setting parameters for trial simulation
n_patients_per_group = 19
mean_initial_pain = 5.75 # mean pain score from ethnographic data
std_dev_initial_pain = 3.20 # standard deviation of pain scores from ethnographic data
effect_size = 0.95 # effect size from ethnographic data
n_simulations = 1000

# Estimated average pain reduction for each SOC treatment - check with matthew 
pain_reduction_CBT = 1.0
pain_reduction_physiotherapy = 0.8
pain_reduction_prescription = 1.5
pain_reduction_support_group = 0.5


### Next I'll write a function to simulate one trial ###

In [51]:


def simulate_trial(n_patients, mean_pain, std_dev, effect_size):
    # Generate initial pain scores
    initial_pain_scores = np.random.normal(mean_pain, std_dev, n_patients * 2).round().astype(int)
    initial_pain_scores = np.clip(initial_pain_scores, 1, 10)
    # print("Initial Pain Scores:", initial_pain_scores)
    
    # Assign SOC treatments
    CBT = np.random.choice([0, 1], size=n_patients * 2)
    physiotherapy = np.random.choice([0, 1], size=n_patients * 2)
    prescription = np.random.choice([0, 1], size=n_patients * 2)
    support_group = np.random.choice([0, 1], size=n_patients * 2)
    # print("SOC Treatments Assigned (CBT, Physiotherapy, Prescription, Support Group):", CBT, physiotherapy, prescription, support_group)
    
    # Calculate SOC effect
    SOC_effect = (CBT * pain_reduction_CBT +
                  physiotherapy * pain_reduction_physiotherapy +
                  prescription * pain_reduction_prescription +
                  support_group * pain_reduction_support_group)
    # print("SOC Effect:", SOC_effect)
    
    # Apply SOC effect to initial pain scores
    adjusted_pain_scores = initial_pain_scores - SOC_effect
    adjusted_pain_scores = np.clip(adjusted_pain_scores, 1, 10)
#   print("Adjusted Pain Scores:", adjusted_pain_scores)
    
    # Ensure the length of adjusted_pain_scores is correct
    if len(adjusted_pain_scores) != n_patients * 2:
        # print("Error: Length of adjusted_pain_scores is not equal to expected size.")
        return [], []

    # Split into control and experimental groups
    control_group = adjusted_pain_scores[:n_patients]
    experimental_group = adjusted_pain_scores[n_patients:]
    # print("Control Group Pain Scores:", control_group)
    # print("Experimental Group Pain Scores:", experimental_group)
    
    # Apply VR therapy effect to the experimental group
    experimental_group -= int(effect_size * std_dev)
    experimental_group = np.clip(experimental_group, 1, 10)
    # print("Final Experimental Group Pain Scores:", experimental_group)
    
    return control_group, experimental_group


In [52]:
# test the function works

control, experimental = simulate_trial(n_patients_per_group, mean_initial_pain, std_dev_initial_pain, effect_size)
print("Control group final pain scores:", control)
print("Experimental group final pain scores:", experimental)    

Control group final pain scores: [2.  1.3 6.5 1.5 1.  1.  5.  6.8 1.  7.5 4.3 1.  3.2 2.5 1.  1.  4.3 5.2
 1. ]
Experimental group final pain scores: [1.  1.  1.  2.8 4.3 1.  1.  3.5 4.5 3.  1.  1.  6.  1.  1.  4.8 1.  1.
 2.7]


## definitions of variables ##
- mean_control: the mean pain score for the control group in each simulation 
- mean_experimental: the mean pain score for the experimental group in each simulation
- mean_difference: the difference between the mean pain scores of experimental and control groups in each simulation

In [64]:
# Simulate multiple trials
np.random.seed(42)

# Create an empty list to store trial results
results = []
for i in range(n_simulations):
    control, experimental = simulate_trial(n_patients_per_group, mean_initial_pain, std_dev_initial_pain, effect_size)
    mean_control = np.mean(control)
    mean_experimental = np.mean(experimental)
    mean_difference = mean_control - mean_experimental  # Control - Experimental to measure reduction in pain?
    results.append((mean_control, mean_experimental, mean_difference))

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Mean_Control', 'Mean_Experimental', 'Mean_Difference'])
print(results_df.head())

   Mean_Control  Mean_Experimental  Mean_Difference
0      4.078947           1.857895         2.221053
1      3.531579           2.189474         1.342105
2      3.468421           2.389474         1.078947
3      4.910526           2.247368         2.663158
4      4.242105           1.942105         2.300000


In [65]:
results_df

,Mean_Control,Mean_Experimental,Mean_Difference
0,4.078947,1.857895,2.221053
1,3.531579,2.189474,1.342105
2,3.468421,2.389474,1.078947
3,4.910526,2.247368,2.663158
4,4.242105,1.942105,2.300000
...,...,...,...
995,4.131579,2.110526,2.021053
996,4.110526,2.405263,1.705263
997,4.573684,2.136842,2.436842
998,4.010526,2.152632,1.857895


In [66]:
# Calculate summary statistics
summary_stats = results_df.describe()
print(summary_stats)

       Mean_Control  Mean_Experimental  Mean_Difference
count   1000.000000        1000.000000      1000.000000
mean       4.218232           2.198711         2.019521
std        0.597938           0.378628         0.719061
min        2.473684           1.221053        -0.526316
25%        3.809211           1.926316         1.515789
50%        4.215789           2.200000         2.021053
75%        4.610526           2.436842         2.506579
max        6.236842           3.578947         4.010526


In [67]:
# Calculate the number of trials where VR therapy was effective
n_effective_trials = results_df[results_df['Mean_Difference'] > 0].shape[0]
n_trials = results_df.shape[0]  
print(f"Number of effective trials: {n_effective_trials}")

Number of effective trials: 999


 ### Permutation testing ###

In [77]:
# Calculate observed difference in means between experimental and control groups
observed_diff = np.mean(experimental) - np.mean(control)  
print(f"Observed Difference in Means: {observed_diff}")

# Combine data 
combined = np.concatenate([control, experimental])

# Number of permutations 
n_permutations = 10000

# Create an empty array to store permutation results
perm_diffs = np.zeros(n_permutations)

# Permutation test
for i in range(n_permutations):
    np.random.shuffle(combined)
    new_control = combined[:len(control)]
    new_experimental = combined[len(control):]
    permuted_diff = np.mean(new_control) - np.mean(new_experimental)  # Control - Experimental
    perm_diffs[i] = permuted_diff


# Calculate the p-value
p_value = np.sum(np.abs(perm_diffs) >= np.abs(observed_diff)) / n_permutations
print(f"p-value: {p_value}")


Observed Difference in Means: -2.221052631578948
p-value: 0.0028
